# BYOC scikit-learn(scikit-learn 알고리즘 커스텀 컨테이너 빌드하기)

***Note: 본 핸즈온은 SageMaker SDK V2에 대응하도록 코드를 수정했습니다. 만약 SDK V1용 코드가 필요하다면 아래 링크를 참조해 주십시오.***

소스 : https://github.com/awslabs/amazon-sagemaker-examples/tree/master/advanced_functionality/scikit_bring_your_own

Amazon SageMaker에서는 여러분들의 알고리즘을 SageMaker 환경에서 학습과 추론을 할수 있도록 패키징하는 기능을 제공합니다. 본 노트북은 SageMaker 학습과 추론용으로 어떻게 도커 컨테이너 이미지를 만들게 되는지 그 과정을 다룹니다.

알고리즘을 컨테이너로 패키징함으로써, 프로그래밍 언어나, 환경, 프레임워크와 의존관계에 관계엾이, 사실상 어떤 코드든 SageMaker환경에서 실행할 수 있습니다. 

_**Note:**_ 본 예제는 sciikit learn알고리즘을 커스텀 컨테이너로 패키징하는 방법을 다루지만, SageMaker는 사전 빌드된 [scikit container](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/scikit_learn_iris/Scikit-learn%20Estimator%20Example%20With%20Batch%20Transform.ipynb)를 제공합니다. scikit 알고리즘이 필요한 모든 경우에 이 컨테이너를 사용할 것을 권장합니다. 본 예제는 커스텀 컨테이너를 직접 빌드하는 사례로 참고해 주십시오. 

설명없이 코드로 바로 시작하시려면 __도커파일 (코드 시작)__ 섹션으로 바로 이동하십시오.  

## 언제 커스텀 알고리즘 컨테이너를 사용해야 하나요?

여러분의 코드를 SageMaker에서 구동하기 위하여 반드시 컨테이너를 직접 만들어야 하는 것은 아닙니다. 만약 여러분이 사용하는 프레임웤이 Apach MXNet이나 Tensorflow 등일 경우 SageMaker는 해당 프레임워크를 직접 지원하기 때문에 여러분은 알고리즘을 구현하는 파이썬 코드만 제공하고 이를 프레임워크 SDK의 entry point에 전달하여 활용할 수 있습니다. SageMaker에서 지원되는 프레임워크는 정기적으로 추가되고 있고, SageMaker에서 지원하는 범용 머신러닝 환경 리스트를 통해 여러분이 작성하시는 알고리즘의 환경이 SageMaker에서 지원되는 지를 확인할 수 있습니다. 

단, 여러분의 환경이나 프레임워크를 지원하는 SDK가 있다고 하더라도 커스텀 컨테이너를 직접 빌드하는 것이 보다 효과적일 경우도 있습니다. 여러분의 알고리즘 코드가 매우 복잡하거나 추가로 다른 프레임웤를 필요로 하는 경우에는 컨테이너를 직접 빌드하는 것이 적절한 선택일 수도 있습니다. 이런 경우에 해당하는 몇가지 사례는 다음과 같습니다. 

1. 프레임워크의 특정 버전이 지원되지 않는 경우
2. 환경에 의존라이브러리들을 추가로 설치하고 설정하는 경우
3. 기본 환경에서 제공되지 않는 학습/배포 솔루션을 사용하는 경우

커스텀 컨테이너를 이용하면 SageMaker에서 사전 제공하지 않는 환경일 경우에도 SageMaker기반으로 동작하도록 할 수 있습니다. 본 예제에서 그 과정을 살펴보겠습니다. 

## 권한설정

본 노트북은 `SageMakerFullAccess`권한에 추가로 Amazon ECR에 접근하기 위한 권한이 필요합니다. 권한을 추가하는 가장 간단한 방법은 관리형 정책인 `AmazonEC2ContainerRegistryFullAccess`를 노트북 인스턴스가 사용중인 역할(role)에 추가하는 것입니다. 이를 위해 노트북 인스턴스를 재시작할 필요는 없으며 수정 즉시 새로운 권한이 할당 될 것입니다. 

## 샘플 시나리오

본 샘플에서는 [scikit-learn][] 머신러닝 패키지의 [의사결정트리][] 알고리즘을 이용하는 간단한 파이썬 예제를 보여줍니다. 샘플코드는 매우 간단합니다. 대신 SageMaker에서 구현을 위해 어떤 구조를 사용하고 코드의 어떤부분이 수정되어야 하는지에 주로 집중할 것입니다. 

본 예제의 방식은 어떤 프로그래밍언어나 환경에서도 동일합니다. 여러분의 환경에 맞는 다른 HTTP 추론 요청을 처리할 수 있는 도구를 선택하여 구현할 수 있습니다.

본 예제에서는 학습과 추론 실행을 단일 이미지로 사용할 것입니다. 하나의 이미지로 학습과 추론을 실행하는 경우 하나의 이미지만 관리하면 되므로 관리절차가 단순해 집니다. 하지만 실제 요구사항에 따라 이 두 환경의 이미지가 분리될 수도 있을 것입니다. 이 경우 Dockerfile을 분리하고 두 개의 이미지를 운영하게 됩니다. 이미지의 분리와 통합은 실제 환경 요구사항과 개발 및 관리 편의성을 고려하여 결정합니다. 

그리고 SageMaker에서 학습과 추론을 모두 실행하지 않고 이 중 한가지 방식만 이용할 경우에는 해당 기능의 이미지만 빌드하면 됩니다.

[scikit-learn]: http://scikit-learn.org/stable/
[의사결정트리]: http://scikit-learn.org/stable/modules/tree.html

본 예제는 컨테이너를 __빌드__하는 부분과 __활용__하는 부분의 두 파트로 나누어져 있습니다. 

# Part 1: 알고리즘을 SageMaker 환경으로 패키징하고 업로드하기 

### 도커 개요

만약 여러분이 이미 도커 환경이 친숙하다면 이 단계는 건너뛰고 다음 섹션으로 이동하십시오.

많은 데이터 사이언티스트들에게 도커 컨테이너는 새로운 기술입니다. 하지만 도커는 여러분의 소프트웨어를 패키징하여 배포하는 매우 간편한 방법이며 그리 어렵지 않습니다. 

도커는 임의의 코드를 스스로 실행환경을 포함하는 __이미지__로 패키징하는 간단한 방법을 제공합니다. 이미지가 생성되고 나면, 도커 환경에서 이 이미지를 __컨테이너__로 실행하게 됩니다. 컨테이너를 실행하는 것은 컨테이너가 프로그램이 실행될 실행환경을 스스로 생성한다는 것을 제외하면 일반적으로 컴퓨터 머신에서 프로그램을 실행하는 것과 다르지 않습니다. 컨테이너는 호스트 환경에서 다른 컴퓨팅 환경과 격리됩니다. 이것은 이 컨테이너가 어디에 실행되는지에 관계없이 독자적인 실행방식을 구성할 수 있게 만들어 줍니다.

도커는 다음과 같은 이유로 conda나 virtualenv와 같은 환경관리에 비해 매우 강력합니다. 
- 도커 환경은 프로그래밍 언에에 독립적입니다.
- 초기 실행명령과 환경변수까지 전체 운영환경을 포괄합니다.

도커 컨테이너는 가상머신과 유사하지만 보다 경령화되어 있습니다. 예를 들어, 컨테이너에서 실행되는 프로그램은 1초 이내에 시작될 수 있고 물리 또는 가상 서버 인스턴스에서 동시에 실행될 수 있습니다. 

도커는 `Dockerfile`이라는 간단한 파일을 사용하여 이미지가 구성되는 방식을 정의합니다. 그 사례는 아래에 제공됩니다. 여러분은 여러분이 만들거나 또는 다른 사람이 만든 도커 이미지를 기반으로 새로운 도커 이미지를 만들 수 있습니다. 이런 방식은 이미지의 생성을 매우 단순화시켜줍니다.

도커는 이런 유연함과 함께 잘 정리된 컨테이너 설정방식을 제공하여 프로그래밍과 개발 커뮤니테에서 열광적인 환영을 받고 있으며, 최근 [Amazon ECS]나 [Amazon EKS]와 같은 많은 서비스가 이 기술을 기반으로 구현되고 제공되고 있습니다. 

Amazon SageMaker 또한 사용자가 임의의 알고리즘을 학습하고 배포할 때 이 도커를 이용합니다. 

Amazon SageMaker 에서 도커 컨테이너는 학습을 위한 방식과, 학습과는 조금 다른 추론 호스팅의 방식으로 호출(invoke) 됩니다. 다음 섹션에서 SageMaker 환경에서 컨테이너를 어떻게 빌드하는지에 대해 설명합니다. 

도커와 관련한 보다 자세한 내용은 아래 링크를 참고하십시오. 

* [Docker home page](http://www.docker.com)
* [Getting started with Docker](https://docs.docker.com/get-started/)
* [Dockerfile reference](https://docs.docker.com/engine/reference/builder/)
* [`docker run` reference](https://docs.docker.com/engine/reference/run/)

[Amazon ECS]: https://aws.amazon.com/ecs/
[Amazon EKS]: https://aws.amazon.com/eks/

### Amazon SageMaker 에서 도커 컨테이너 실행 방식 

SageMaker에서 학습과 호스팅에 동일한 이미지를 사용할 수 있으며, 컨테이너를 실행시 `train` 또는 `serve`라는 매개변수(argument)를 이용하여 컨테이너를 실행합니다. 컨테이너가 이 매개변수를 처리하는 방식은 컨테이너 구성에 따라 달라집니다.

* 본 샘플에서는 도커파일(Dokerfile)에서 `ENTRYPOINT`를 사용하지 않습니다. 대신 학습시점에는 [`train`](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html) 명령을 서빙(호스팅)시점에는 [`serve`](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html) 명령을 이용하여 도커를 실행합니다.
* 만약 도커파일에서 `ENTRYPOINT`를 이용하여 프로그램을 명시하게 되면 도커 실행시 해당프로그램이 실행되며 `train` 또는 `serve`가 첫번째 매개변수로 전달될 것입니다. 프로그램은 이 매개변수에 따라 무슨 동작을 해야할 지를 판단할 수 있습니다. 
* 만약 학습과 호스팅 컨테이너를 분리하는 경우 (또는 둘 장 하나의 방식만 활용하는 경우) 도커파일에 `ENTRYPOINT`에 프로그램을 정의할 수 있고, 이 때 전달되는 첫번째 매개변수는 무시하거나 또는 검증용으로 사용할 수 있습니다. 


#### 학습(training)용으로 커스텀 컨테이너 이용하기 

SageMaker에서 학습이 실행될 때 여러분의 `train` 스크립트가 실행됩니다. 일반적인 파이썬 프로그램이 실행되는 것과 마찬가지 방식이며, `/opt/ml` 디렉토리의 몇가지 파일들이 활용될 수 있습니다.

    /opt/ml
    |-- input
    |   |-- config
    |   |   |-- hyperparameters.json
    |   |   `-- resourceConfig.json
    |   `-- data
    |       `-- <channel_name>
    |           `-- <input data>
    |-- model
    |   `-- <model files>
    `-- output
        `-- failure

##### 데이터 입력

* `/opt/ml/input/config` - 프로그램이 실행되는 방식을 관리할 수 있는 정보가 포함됩니다. `hyperparameters.json` 은 하이퍼파라미터 이름과 값을 가지는 JSON 형식의 딕셔너리입니다. 값은 string 형태로 전달되며 필요시 타입을 변환하여 사용합니다. `resourceConfig.json` 은 분산학습에서 네트워크 레이아웃을 알려주는 JSON형식의 파일입니다. 

* `/opt/ml/input/data/<channel_name>/` - (파일 모드일 경우) 채널에 대한 입력데이터가 저장됩니다. 채널은 CreateTrainingJob(또는 fit)을 호출할 때 생성되며 알고리즘 코드에서 동일하게 사용해야 합니다. 알고리즘에서 사용할 파일들이 S3로부터 지정한 채널로 복사되며 S3 키의 트리 구조를 유지합니다. 

* `/opt/ml/input/data/<channel_name>_<epoch_number>` (파이프 모드일 경우) 실행되는 epoch을 위한 파이프입니다. epoch은 0에서부터 이 채널을 읽을때마다 증가합니다. epoch숫자에 제한은 없으며 다음 epoch을 실행하기 전에 pipe를 close 해야 합니다. 


##### 학습결과 출력

* `/opt/ml/model/` - 알고리즘이 생성하는 모델이 저장되는 디렉토리입니다. 모델의 형식은 여러분이 지정하는 방식에 따라 달라지며, 단일 파일일 수도 있고 트리구조를 가지는 디렉토리 전체일 수도 있습니다. 모델 파일은 `DescribeTrainingJob` 호출 결과로 리턴되는 지정된 S3 위치에 사용가능하도록 export 됩니다. 

* `/opt/ml/output` - 작업이 실패할 때 `failure` 파일을 저장하는 디렉토리입니다. 파일의 내용은 `DescribeTrainingJob`호출시 `FailureReason` 필드의 값으로 리턴됩니다. 작업이 성공적으로 종료될 경우 이 파일은 필요가 없으므로 무시됩니다. 

#### 호스팅(추론)용으로 컨테이너 이용하기

추론 호스팅은 HTTP를 통해 들어오는 추론 요청(request)에 응답해야 하므로 학습과는 다른 모델을 가집니다. 본 사례에서는 추론 처리를 위한 일반적인 권장 파이썬 서빙 스택을 이용할 것입니다.

![Request serving stack](stack.png)

대부분의 경우에는 본 예제에 샘플로 구현된 스택을 그대로 사용하실 수 있을 것입니다. 

Amazon SageMaker기반 추론을 위해 컨테이너 내부에서 다음 두가지 URL을 사용합니다. 

* `/ping` - 인프라로부터 `GET` 요청을 받아서 처리합니다. 컨테이너가 정상적으로 동작하고 요청을 받을 수 있는 경우 200을 리턴합니다.
* `/invocations` - 추론 클라이언트로부터 `POST`요청을 받아 처리합니다. 요청과 응답의 형식은 알고리즘에 따라 달라집니다. 클라이언트가 `ContentType`과 `Accept` 헤더를 지정한 경우 함께 전달됩니다.

추론 컨테이너의 모델 파일 위치는 학습에서 생성한 모델을 저장할 때 사용한 위치와 동일합니다. 

    /opt/ml
    `-- model
        `-- <model files>

### The parts of the sample container

본 예제의 `container` 디렉토리에 샘플 알고리즘을 SageMaker환경으로 패키징하는 모든 파일들이 저장되어 있습니다.

    .
    |-- Dockerfile
    |-- build_and_push.sh
    `-- decision_trees
        |-- nginx.conf
        |-- predictor.py
        |-- serve
        |-- train
        `-- wsgi.py

이들 파일을 차례로 살펴보면:

* __`Dockerfile`__ - 도커 컨테이너 이미지를 빌드하는 방법이 정의됩니다. 다음 섹션에서 다시 설명합니다. 
* __`build_and_push.sh`__ - Dockerfile을 이용하여 컨테이너 이미지를 빌드하고 ECR로 push하는 스크립트입니다. 본 노트북에서 이 쉘을 직접 실행할 것입니다. 이후 여러분의 알고리즘에 적용할 경우에도 이 파일을 그대로 사용할 수 있습니다. 
* __`decision_trees`__ - 컨테이너 내부로 복제될 파일들이 저장되어 있습니다. 
* __`local_test`__ - SageMaker 노트북 등 로컬 환경에서 도커를 구동하고 테스트하는 방법을 보여줍니다. 이 방법을 이용하면 SageMaker 컨테이너 환경을 이용하기 전에 작은 데이터셋으로 빠르게 실행하여 결함을 찾을 수 있습니다. 본 예제에서 사용방법을 다룰 것입니다. 

본 예제에서 우리는 5개의 파일을 컨테이너로 복제할 것입니다. 실제로는 여러분의 유즈케이스에 따라 이정도의 파일이 충분할 수도 있고 또는 더 많은 파일이 필요할 수도 있습니다. 하지만 이 5개의 파일이 커스텀 파이썬 컨테이너의 표준 구조가 됩니다. 예제코드와 다른 프로그래밍 언어나 도구(toolkit)를 사용할 경우에는 다른 구성을 가질 것입니다. 

본 예제에서 컨테이너에 복제될 파일은 다음과 같습니다. (`cifar10`디렉토리 내부의 파일들)

* __`cifar10.py`__ - 알고리즘의 실행을 구현하는 프로그램 코드입니다. 
* __`resnet_model.py`__ - Resnet 모델을 정의하는 코드입니다. (`cifar10.py`에서 사용합니다.)


* __`nginx.conf`__ - nginx front-end를 구성하는 설정 파일입니다. 일반적으로 제공되는 파일을 그대로 사용가능합니다.
* __`predictor.py`__ - Flask 웹서버를 로직을 구현하는 프로그램입니다. 실제 운영환경에 여러분의 애플리케이션을 적용할 때에는 이 부분을 커스터마아징하게 될 것입니다. 본 예제의 로직은 복잡하지 않기 때문에 하나의 파일로 구현하지만, 실제로는 커스텀로직의 내용에 따라 여러 파일로 분리될 수도 있을 것입니다.
* __`serve`__ - 컨테이너가 추론 호스팅을 할 때 실행되는 프로그램입니다. `predictor.py`에 구현된 Flask 웹서버 인스턴스를 복수로 실행하는 gunicorn 서버를 구동합니다. 일반적으로 본 예제에서 제공되는 파일을 그대로 사용가능합니다. 
* __`train`__ - 컨테이너가 학습을 진행할 때 실행되는 프로그램입니다. 학습 알고리즘을 구현하기 위해 이 부분을 수정할 것입니다. 
* __`wsgi.py`__ - Flask 애플리케이션을 구동하기 위한 간단한 wrapper입니다. 일반적으로 본 예제에서 제공된 파일을 그대로 사용가능합니다.


요약하면, 이후 여러분의 실제 애플리케이션에 여러분의 알고리즘 실행코드를 적용할 때에는 해당 코드와 함께 `train`과 `predictor.py` 부분을 변경하게 됩니다. 

### 도커파일 (코드 시작)

Dockerfile은 빌드할 이미지를 정의합니다. 실행할 시스템의 운영체제와 환경을 완전히 정의하는 것으로 생각해도 좋습니다. 하지만 실제로 도커 컨테이너의 실행은 운영체제를 모두 준비하는 것에 비해 매우 경량화되어 있으며 기초 동작은 호스트머신의 Linux를 활용합니다. 

파이선 데이터사이언스 스택을 준비하기 위해 표준 Ubunto 이미지로 부터 시작하하여 기본 도구와 scikit-learn을 설치하겠습니다. 그리고 이 환경에서 실행될 사용자 알고리즘 코드를 추가하겠습니다. 

아래 코드를 이용하여 Dockerfile을 살펴봅니다.

In [1]:
# docker

In [112]:
!pwd

/home/ec2-user/SageMaker/sagemaker-byos-byoc/BYOC/scikit_bring_your_own_kr


In [113]:
! ls -al /home/ec2-user/

total 2936
drwx------ 23 ec2-user ec2-user    4096 Apr  2 00:21 .
drwxr-xr-x  5 root     root        4096 Mar 23 16:48 ..
drwxrwxr-x 27 ec2-user ec2-user    4096 Mar  6 05:39 anaconda3
drwxr-xr-x  2 ec2-user root        4096 Mar 31 07:45 .aws
-rw-r--r--  1 ec2-user ec2-user      18 Aug 30  2017 .bash_logout
-rw-r--r--  1 ec2-user ec2-user     193 Aug 30  2017 .bash_profile
-rw-rw-r--  1 ec2-user ec2-user     658 Mar  6 05:36 .bashrc
drwxr-xr-x  5 ec2-user ec2-user    4096 Apr  1 02:32 .cache
drwxrwxr-x  2 ec2-user ec2-user    4096 Mar  6 05:36 .conda
-rw-rw-r--  1 ec2-user ec2-user     148 Mar 23 16:35 .condarc
drwxrwxr-x  5 ec2-user ec2-user    4096 Mar 23 16:49 .config
-rw-r--r--  1 ec2-user ec2-user       0 Mar 23 16:49 .cw-agent-baked
-rw-rw-r--  1 ec2-user ec2-user    1201 Mar  6 05:36 .dlamirc
drwxrwxr-x  8 ec2-user ec2-user    4096 Mar 31 08:12 .dl_binaries
drwx------  2 ec2-user ec2-user    4096 Apr  2 02:33 .docker
drwxrwxr-x 10 ec2-user ec2-user    4096 Mar  6 05:29 examples


In [114]:
! cat /home/ec2-user/.bash_profile

# .bash_profile

# Get the aliases and functions
if [ -f ~/.bashrc ]; then
	. ~/.bashrc
fi

# User specific environment and startup programs

PATH=$PATH:$HOME/.local/bin:$HOME/bin

export PATH


In [115]:
! cat /home/ec2-user/.bashrc

#Source .dlamirc in .bashrc
source ~/.dlamirc

# .bashrc

# Source global definitions
if [ -f /etc/bashrc ]; then
	. /etc/bashrc
fi

# User specific aliases and functions

# >>> conda initialize >>>
# !! Contents within this block are managed by 'conda init' !!
__conda_setup="$('/home/ec2-user/anaconda3/bin/conda' 'shell.bash' 'hook' 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__conda_setup"
else
    if [ -f "/home/ec2-user/anaconda3/etc/profile.d/conda.sh" ]; then
        . "/home/ec2-user/anaconda3/etc/profile.d/conda.sh"
    else
        export PATH="/home/ec2-user/anaconda3/bin:$PATH"
    fi
fi
unset __conda_setup
# <<< conda initialize <<<



In [116]:
! ls /opt/ml/metadata

resource-metadata.json


In [117]:
! ls container/

build_and_push.sh  decision_trees  Dockerfile  local_test  ReadMe.md


In [118]:
! cat container/ReadMe.md

# Bring-your-own Algorithm Sample

This example shows how to package an algorithm for use with SageMaker. We have chosen a simple [scikit-learn][skl] implementation of decision trees to illustrate the procedure.

SageMaker supports two execution modes: _training_ where the algorithm uses input data to train a new model and _serving_ where the algorithm accepts HTTP requests and uses the previously trained model to do an inference (also called "scoring", "prediction", or "transformation").

The algorithm that we have built here supports both training and scoring in SageMaker with the same container image. It is perfectly reasonable to build an algorithm that supports only training _or_ scoring as well as to build an algorithm that has separate container images for training and scoring.v

In order to build a production grade inference server into the container, we use the following stack to make the implementer's job simple:

1. __[nginx][nginx]__ is a light-weight layer that handles the

In [119]:
! cat /opt/ml/metadata/resource-metadata.json

{"ResourceArn": "arn:aws:sagemaker:ap-northeast-2:533821149268:notebook-instance/test2", "ResourceName": "test2"}

In [120]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 3 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:18.04

MAINTAINER Amazon AI <sage-learner@amazon.com>


RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python3-pip \
         python3-setuptools \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

RUN ln -s /usr/bin/python3 /usr/bin/python
RUN ln -s /usr/bin/pip3 /usr/bin/pip

# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.
RUN pip --no-cache-dir install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gunicorn

# Set some environment variabl

In [121]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 3 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:18.04

MAINTAINER Amazon AI <sage-learner@amazon.com>


RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python3-pip \
         python3-setuptools \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

RUN ln -s /usr/bin/python3 /usr/bin/python
RUN ln -s /usr/bin/pip3 /usr/bin/pip

# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.
RUN pip --no-cache-dir install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gunicorn

# Set some environment variabl

In [185]:
#!cat container/Dockerfile-random-forest

### 컨테이너 빌드 및 등록

다음 쉘스크립트는 `docker build` 명령을 이용하여 컨테이너를 빌드하고 `docker push`명령을 이용하여 ECR에 빌드한 이미지를 push하는 방법을 보여줍니다. 해당 코드는 `container/build-and-push.sh`의 내용과 동일하며 `decision_trees_sample`라는 이름으로 이미지를 빌드하고 push 하고자 할 때 `build-and-push.sh decision_trees_sample`와 같은 형식으로 실행하면 됩니다.

아래 코드는 여러분의 어카운트의 디폴트 리전 (또는 SageMaker 노트북 인스턴스를 사용중인 경우 노트북이 생성된 리전)에서 ECR 레포지토리를 찾고, 만약 레포지토리가 없다면 이를 생성할 것입니다. 

# decision-tree

In [12]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-decision-trees # repository name
folder_name=decision_trees
env_param="FOLDER_DIR=${folder_name}"
cd container

chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
#region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  --build-arg ${env_param} -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}


docker push ${fullname}

Login Succeeded

Step 1/12 : FROM ubuntu:18.04
 ---> 3339fde08fc3
Step 2/12 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 1302b3fdc513
Step 3/12 : ARG FOLDER_DIR
 ---> Using cache
 ---> 2a235613cb63
Step 4/12 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python3-pip          python3-setuptools          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Running in 5c7f545b9c48
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [348 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1402 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic/main amd

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [363]:
! ls -al

total 192
drwxrwxr-x 5 ec2-user ec2-user   4096 Apr  5 02:58 .
drwxrwxr-x 4 ec2-user ec2-user   4096 Mar 30 08:21 ..
drwxrwxr-x 7 ec2-user ec2-user   4096 Apr  5 02:46 container
drwxrwxr-x 2 ec2-user ec2-user   4096 Mar 30 08:21 data
drwxrwxr-x 2 ec2-user ec2-user   4096 Mar 31 08:15 .ipynb_checkpoints
-rw-rw-r-- 1 ec2-user ec2-user 158705 Apr  5 02:58 scikit_bring_your_own.ipynb
-rw-rw-r-- 1 ec2-user ec2-user  13595 Mar 30 08:21 stack.png


In [356]:
! docker build --help


Usage:	docker build [OPTIONS] PATH | URL | -

Build an image from a Dockerfile

Options:
      --add-host list           Add a custom host-to-IP mapping (host:ip)
      --build-arg list          Set build-time variables
      --cache-from strings      Images to consider as cache sources
      --cgroup-parent string    Optional parent cgroup for the container
      --compress                Compress the build context using gzip
      --cpu-period int          Limit the CPU CFS (Completely Fair
                                Scheduler) period
      --cpu-quota int           Limit the CPU CFS (Completely Fair
                                Scheduler) quota
  -c, --cpu-shares int          CPU shares (relative weight)
      --cpuset-cpus string      CPUs in which to allow execution (0-3, 0,1)
      --cpuset-mems string      MEMs in which to allow execution (0-3, 0,1)
      --disable-content-trust   Skip image verification (default true)
  -f, --file string             Name of the Dockerf

# other model=sklearn

In [16]:
#! ls -l ./container/decision_trees

#! ls -l ./container/sagemaker-random-forest

In [3]:
# %%sh

# # The name of our algorithm
# algorithm_name=sagemaker-random-forest # repository name
# folder_name=random_forest
# env_param="FOLDER_DIR=${folder_name}"
# echo $env_param

# cd container

# chmod +x ${folder_name}/train
# chmod +x ${folder_name}/serve

# account=$(aws sts get-caller-identity --query Account --output text)

# # Get the region defined in the current configuration (default to us-west-2 if none defined)
# region=$(aws configure get region)
# #region=${region:-us-west-2}

# fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"
# echo ${fullname}
# # If the repository doesn't exist in ECR, create it.
# aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

# if [ $? -ne 0 ]
# then
#     aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
# fi

# # Get the login command from ECR and execute it directly
# aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# # Build the docker image locally with the image name and then push it to ECR
# # with the full name.
# #docker build  --build-arg ${env_param} -t ${algorithm_name} .
# docker build  --build-arg ${env_param} -t ${algorithm_name} .

# docker tag ${algorithm_name} ${fullname}

# echo $fullname
# docker push ${fullname}

FOLDER_DIR=random_forest
533821149268.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-random-forest:latest
Login Succeeded

Step 1/12 : FROM ubuntu:18.04
 ---> 3339fde08fc3
Step 2/12 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 1302b3fdc513
Step 3/12 : ARG FOLDER_DIR
 ---> Using cache
 ---> 2a235613cb63
Step 4/12 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python3-pip          python3-setuptools          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> abd7276cc2eb
Step 5/12 : RUN ln -s /usr/bin/python3 /usr/bin/python
 ---> Using cache
 ---> 634116fae77b
Step 6/12 : RUN ln -s /usr/bin/pip3 /usr/bin/pip
 ---> Using cache
 ---> 331953b454c6
Step 7/12 : RUN pip --no-cache-dir install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gunicorn
 ---> Using cache
 ---> 05645129e6d7
Step 8/12 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> d5e90c10fb57
Step 9/12 

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [ ]:
# %%sh

# # The name of our algorithm
# algorithm_name=sagemaker-decision_tree # repository name
# folder_name=random_forest
# env_param="FOLDER_DIR=${folder_name}"
# echo $env_param

# cd container

# chmod +x ${folder_name}/train
# chmod +x ${folder_name}/serve

# account=$(aws sts get-caller-identity --query Account --output text)

# # Get the region defined in the current configuration (default to us-west-2 if none defined)
# region=$(aws configure get region)
# #region=${region:-us-west-2}

# fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"
# echo ${fullname}
# # If the repository doesn't exist in ECR, create it.
# aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

# if [ $? -ne 0 ]
# then
#     aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
# fi

# # Get the login command from ECR and execute it directly
# aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# # Build the docker image locally with the image name and then push it to ECR
# # with the full name.
# docker build  --build-arg ${env_param} -t ${algorithm_name} .

# docker tag ${algorithm_name} ${fullname}

# echo $fullname
# docker push ${fullname}

In [ ]:
FOLDER_DIR

In [17]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-random-forest # repository name
env_param="FOLDER_DIR=$algorithm_name"
echo $env_param

cd container

chmod +x ${algorithm_name}/train
chmod +x ${algorithm_name}/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
#region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"
echo ${fullname}
# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build  --build-arg ${env_param} -t ${algorithm_name} .

echo ${env_param}
docker tag ${algorithm_name} ${fullname}

echo $fullname
docker push ${fullname}

FOLDER_DIR=sagemaker-random-forest
533821149268.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-random-forest:latest
Login Succeeded

Step 1/12 : FROM ubuntu:18.04
 ---> 3339fde08fc3
Step 2/12 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 1302b3fdc513
Step 3/12 : ARG FOLDER_DIR
 ---> Using cache
 ---> 2a235613cb63
Step 4/12 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python3-pip          python3-setuptools          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Running in 58d9cdb2d613
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [24.5 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2045 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://arch

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [ ]:
# %%sh

# # The name of our algorithm
# algorithm_name=sagemaker-xgboost # repository name

# cd container

# chmod +x decision_trees/train
# chmod +x decision_trees/serve

# account=$(aws sts get-caller-identity --query Account --output text)

# # Get the region defined in the current configuration (default to us-west-2 if none defined)
# region=$(aws configure get region)
# #region=${region:-us-west-2}

# fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# # If the repository doesn't exist in ECR, create it.
# aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

# if [ $? -ne 0 ]
# then
#     aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
# fi

# # Get the login command from ECR and execute it directly
# aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# # Build the docker image locally with the image name and then push it to ECR
# # with the full name.

# docker build  -t ${algorithm_name} .
# docker tag ${algorithm_name} ${fullname}

# docker push ${fullname}

## 로컬 머신 (또는 SageMaker 노트북 인스턴스)에서 알고리즘 테스트 

알고리즘을 처음 패키징할 때에는 여러분의 코드가 잘 동작하는지 테스트가 필요할 것입니다. `container/local_test`디렉토리에 이를 위한 프레임워크가 저장되어 있습니다. 해당 폴더에는 다음 세 가지 스크립트가 제공되며 위에 설명된 구조와 유사하게 컨테이너와 디렉토리 구조를 사용할 수 있도록 해 줍니다.

* `train_local.sh`: 이미지 이름과 함께 호출하면 로컬에서 학습(training)을 실행할 수 있습니다. 예를 들어 다음과 같이 호출합니다.
```bash
$ ./train_local.sh sagemaker-decision-trees
```  
실행이 완료되면 `test_dir/model`디렉토리에 모델을 생성할 것입니다. `test_dir/input/data/...` 와 `input/config/hyperparameters.json`에는 각각 학습용 데이터와 하아퍼파라미터 설정파일이 저장되어 있습니다. 필요시 해당 내용을 수정하여 테스트합니다. (이 파일 경로는 이를 사용하는 train 파이썬코드의 내용과 매치되어야 합니다. `train_local.sh` 쉘스크립트를 직접 열어보시면 `docker run`을 통해 train 프로세스를 실행하는 간단한 코드임을 확인할 수 있습니다.)

* `serve_local.sh`: 학습이 완료된 후 호스팅을 위해 로컬에서 serve를 실행하는 스크립트입니다. 로컬 학습에서 실행한 것과 마찬가지로 이미지이름을 주고 호출합니다. 예를 들어 다음과 같이 호출합니다.
```bash
$ ./serve_local.sh sagemaker-decision-trees
```
위 코드를 실행하면 8080포트를 통해 외부요청을 받을 수 있도록 컨테이너 프로세스가 구동하고 추론을 위한 `test_dir/model`에 저장된 학습된 의사결정트리 모델파일을 로드합니다. 키보드 인터럽트를 보내면 프로세스를 멈출 수 있습니다. 

* `predict.sh`: 예측용 입력데이터와 함께 로컬 엔드포인트로 추론 http 요청을 보냅니다. content type은 디폴트 `text/csv`를 사용합니다. 예를 들어 다음과 같이 호출합니다.
```bash
$ ./predict.sh payload.csv text/csv
```

본 노트북과 함께 제공된 디렉토리는 지금 다루고 있는 의사결정트리 샘플을 테스트하는 용도로 셋업된 상태입니다. local_test 디렉토리의 test_dir 폴더의 구조를 함께 참고하십시오. 


In [ ]:
# train_local.sh

# Part 2: Amazon SageMaker에서 커스텀 알고리즘으로 학습과 추론 실행

컨테이너 패키징이 완료되면 이제 SageMaker에서도 학습과 추론을 실행할 수 있습니다. 앞서 만든 알고리즘 컨테이너를 그대로 사용합니다. 

## 환경 셋업

SageMaker에서 사용할 S3 버킷을 설정합니다. 

In [28]:
import string, random
string_pool = string.ascii_lowercase + string.ascii_uppercase + string.digits
result = "" 
for i in range(8) : 
    result += random.choice(string_pool)

In [29]:
#result= 'random_forest'

In [30]:
# S3 prefix
prefix = f'DEMO-scikit-byo-iris-{result}'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role() # role이 s3 접근 권한 이던가? 

## 세션 생성

세션은 SageMaker 환경에 대한 접속 파리미터를 기억합니다. 이후 SageMaker 동작에 이 세션을 사용할 것입니다. 

In [31]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## 학습용 데이터 업로드

SageMaker Python SDK에서 제공되는 도구를 이용하여 데이터를 디폴트 버킷에 업로드합니다.

학습작업이 대용량 데이터를 사용하는 경우 S3 데이터 생성에 Amazon Athena, AWS Glue, or Amazon EMR와 같은 빅데이터 도구들이 함께 사용될 수 있습니다. 본 노트북은 간단히 [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set) 데이터셋을 사용하며 코드와 함께 제공되었습니다. 

In [32]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [23]:
data_location

's3://sagemaker-ap-northeast-2-533821149268/DEMO-scikit-byo-iris-wVg55AlB'

## Estimator 생성 및 학습 실행

SageMaker를 사용하기 위해 컨테이너를 이용하여 학습할 수 있도록 Estimator를 생성합니다. 생성시 SageMaker 학습환경을 위한 설정값들을 지정합니다. 

* __container name__ - 이전 쉘스크립트에서 생성한 이름
* __role__ - 학습을 실행할 IAM 역할(role)
* __instance count__ - 학습에 사용할 머신의 개수
* __instance type__ - 학습에 사용할 머신의 인스턴스 타입
* __output path__ - 학습결과로 생성되는 모델 아티펙트의 저장 위치
* __session__ - 이전 단계에서 지정한 SageMaker 세션 

그 다음 S3에 업로드한 학습용 데이터를 이용하여 `fit()` 명령을 실행합니다.

In [ ]:
#%%time
SHIFT_LEN = 20
# 데이터 입수

def data_input(BUCKET,FILE_NAME,SHIFT_LEN):
    train_df = pd.read_parquet(f's3://{BUCKET}/{FILE_NAME}')       
    #train_df.columns = col_list
    #train_df = train_df[['ISO_YEAR_WEEK','REAL_SO_BOX']]
    #train_df.set_index('ISO_YEAR_WEEK',inplace=True)
    test_len = 4
    display(train_df)
    train_df.info()
    #train_df = train_df[train_df['ITEM_CD']=='1000043']    
    train_df = train_df[train_df['ITEM_CD']=='1000043']    
    
    test_output = train_df[-(SHIFT_LEN+test_len):]
    train_output = train_df[:-test_len]
    return train_output,test_output



# def data_input(FILE_NAME):
#     train_df = pd.read_parquet(FILE_NAME)   
#     #train_df.columns = col_list
#     #train_df = train_df[['ISO_YEAR_WEEK','REAL_SO_BOX']]
#     #train_df.set_index('ISO_YEAR_WEEK',inplace=True)
#     test_len = 4
#     train_df[train_df['ITEM_CD']=='1000043']
#     test_output = train_df[-(SHIFT_LEN+test_len):]
#     train_output = train_df[:-test_len]
#     return train_output,test_output

train_df,test_df = data_input(BUCKET,FILE_NAME,20)
#train_df,test_df = data_input(FILE_NAME)

train_df

# 컬럼 선택 및 인코딩
# shift(1,2,3,4) 1년차 코드를 기준으로 def 선언

# 컬럼 선택 및 원핫 인코딩
def common_processing(train_df):
    train_df = train_df.set_index('ISO_YEAR_WEEK')

    del train_df['PLAN_WEEK_ID']
    del train_df['ITEM_CD']
    train_df_dtypes = train_df.dtypes
    to_one_hot_columns = list(train_df_dtypes[train_df_dtypes == 'object'].index)
    #to_one_hot_columns.extend(['PL1_GB','PL2_GB','PL3_GB'])
    to_one_hot = train_df[to_one_hot_columns]
    one_hot = pd.get_dummies(to_one_hot)
    train_df.drop(columns=to_one_hot_columns, inplace=True)
    result = pd.concat([train_df,one_hot],axis=1)
    return result

train_df = common_processing(train_df)
test_df = common_processing(test_df)

train_df

# sb
STEP=1 

test_df = common_processing(test_df)

from IPython.display import display

def train_df_shifting(train_df,STEP, SHIFT_LEN):
    SHIFT_LEN = SHIFT_LEN+1
    df_list = []
    df_list.append(train_df)
    for SHIFT_STEP in range(STEP,SHIFT_LEN):
        train_df_shift = pd.DataFrame(train_df['REAL_SO_BOX'].shift(SHIFT_STEP))
        train_df_shift.columns = [f'SHIFT_STEP{SHIFT_STEP}']
        df_list.append(train_df_shift)        
    result = pd.concat(df_list,axis=1)
    result.dropna(inplace=True)
    return result       

#202016 까지 사용가능함
train_shift = train_df_shifting(train_df,STEP,SHIFT_LEN)

test_shift = train_df_shifting(test_df,STEP,SHIFT_LEN)

def match_col_train_test(train_shift,test_shift):
    train_shift_columns = train_shift.columns
    missing_cols = set( train_shift_columns ) - set( test_shift.columns )
    # Add a missing column in test set with default value equal to 0
    for col in missing_cols:
        test_shift[col] = 0
    # Ensure the order of column in the test set is in the same order than in train set
    test_shift = test_shift[train_shift_columns]
    return test_shift

test_shift = match_col_train_test(train_shift,test_shift)

def split_xy(train_shift):
    X = train_shift.copy()
    X.drop(['REAL_SO_BOX'], axis=1, inplace=True)
    y = train_shift['REAL_SO_BOX'].copy()
    return X,y
X, y = split_xy(train_shift)
print((train_shift.shape, X.shape, y.shape))
feature_dir_local = 'trnval'

# if not os.path.exists(feature_dir_local):
#     os.mkdir(feature_dir_local)

test_X, test_y = split_xy(test_shift)

In [13]:
hyperparameter = {'max_depths':3,'random_state':4000}

In [14]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-decision-trees:latest'.format(account, region)

# tree = sage.estimator.Estimator(role=role,
#                                 instance_count=1,
#                                 instance_type='ml.m4.xlarge',
#                                 image_uri=image,
#                                 output_path="s3://{}/output".format(sess.default_bucket()),
#                                 sagemaker_session=sess,
#                                hyperparameters= hyperparameter)

tree = sage.estimator.Estimator(role=role,
                                instance_count=1,
                                instance_type='ml.m4.xlarge',
                                image_uri=image,
                                output_path="s3://{}/output".format(sess.default_bucket()),
                                sagemaker_session=sess,
                               hyperparameters= hyperparameter)

In [354]:
image

'533821149268.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-decision-trees:latest'

In [15]:
tree.fit(data_location)

2021-04-05 03:07:28 Starting - Starting the training job...
2021-04-05 03:07:51 Starting - Launching requested ML instancesProfilerReport-1617592048: InProgress
......
2021-04-05 03:08:51 Starting - Preparing the instances for training......
2021-04-05 03:09:52 Downloading - Downloading input data...
2021-04-05 03:10:18 Training - Downloading the training image...
2021-04-05 03:10:55 Uploading - Uploading generated training modelStarting the training.
hyperparameters: {'max_depths': '3'}
Training complete.

2021-04-05 03:11:12 Completed - Training job completed
Training seconds: 84
Billable seconds: 84


In [33]:
data_location

's3://sagemaker-ap-northeast-2-533821149268/DEMO-scikit-byo-iris-YBeK6fBY'

In [25]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-random-forest:latest'.format(account, region)

tree2 = sage.estimator.Estimator(role=role,
                                instance_count=1,
                                instance_type='ml.m4.xlarge',
                                image_uri=image,
                                output_path="s3://{}/output".format(sess.default_bucket()),
                                sagemaker_session=sess)

In [26]:
tree2.fit(data_location)

2021-04-05 04:18:58 Starting - Starting the training job...
2021-04-05 04:19:00 Starting - Launching requested ML instancesProfilerReport-1617596338: InProgress
......
2021-04-05 04:20:08 Starting - Preparing the instances for training......
2021-04-05 04:21:18 Downloading - Downloading input data
2021-04-05 04:21:18 Training - Downloading the training image...
2021-04-05 04:21:59 Uploading - Uploading generated training model
2021-04-05 04:21:59 Failed - Training job failed
Starting the training.
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
Exception during training: could not convert string to float: 'setosa'
Traceback (most recent call last):
  File "/opt/program/train", line 64, in train
    clf = clf.fit(train_X, train_y)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py", line 

UnexpectedStatusException: Error for Training job sagemaker-random-forest-2021-04-05-04-18-58-728: Failed. Reason: AlgorithmError: Exception during training: could not convert string to float: 'setosa'
Traceback (most recent call last):
  File "/opt/program/train", line 64, in train
    clf = clf.fit(train_X, train_y)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py", line 279, in fit
    y = np.ascontiguousarray(y, dtype=DOUBLE)
  File "/usr/local/lib/python3.6/dist-packages/numpy/core/numeric.py", line 632, in ascontiguousarray
    return array(a, dtype, copy=False, order='C', ndmin=1)
ValueError: could not convert string to float: 'setosa'


In [104]:
! cat /opt/ml/metadata/resource-metadata.json

{"ResourceArn": "arn:aws:sagemaker:ap-northeast-2:533821149268:notebook-instance/test2", "ResourceName": "test2"}

## 모델 호스팅(추론용)

학습된 모델을 활용하여 HTTP 엔드포인트를 만들고 실시간 예측 서비스를 실행합니다. 다음 단계로 진행합니다.

### 모델 배포(deploy)

학습이 완료된 모델로부터 `deploy` 명령을 호출하면 SageMaker에 호스팅환경에 배포됩니다. 명령 실행시 인스턴스 개수와 타입을 (그리고 선택적으로 직렬화, 역직렬화 함수를) 지정합니다. 

In [ ]:
%%time
from sagemaker.serializers import CSVSerializer
predictor = tree.deploy(1, 'ml.m4.xlarge', serializer=CSVSerializer())

### 예측 실행 

학습데이터중 일부를 샘플링하여 예측용 데이터를 만들겠습니다. (학습에 사용한 데이터를 테스트용으로 사용하는 것은 바람직하지 않을 수 있지만 알고리즘 동작확인을 목적으로 진행하겠습니다.)

In [61]:
shape = pd.read_csv("data/iris.csv", header=None)
shape.sample(3)

,0,1,2,3,4
44,setosa,5.1,3.8,1.9,0.4
31,setosa,5.4,3.4,1.5,0.4
66,versicolor,5.6,3.0,4.5,1.5


In [12]:
import itertools

a = [50*i for i in range(3)]
b = [40+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data = shape.iloc[indices[:-1]]

`deploy()`의 결과로 리턴받은 `predictor`의 `predict()` 함수를 호출하여 예측하고자 하는 데이터에 대한 예측결과를 쉽게 얻을 수 있습니다. 직렬화함수는 데이터 컨버젼을 처리해 줍니다. 

In [13]:
print(predictor.predict(test_data.values).decode('utf-8'))

setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica



### (선택) 리소스 삭제

엔드포인트의 활용이 끝나면 리소스를 삭제합니다. 

In [14]:
sess.delete_endpoint(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## 배치 추론 작업 실행 

대량의 데이터에 대한 배치 추론을 위해 [Amazon SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)을 이용할 수 있습니다. 배치작업은 입력데이터의 S3 경로와 결과를 저장할 S3 출럭폴더의 경로를 전달받습니다. 본 예제에서는 엔드포인트 호스팅과 유사하게 학습데이터셋을 이용하여 배치 추론을 진행해 보겠습니다. 

### 배치변환(Transform)작업 생성

추론 작업을 실행할 컨테이너를 정의하기 위해 `Transformer`를 생성합니다. `Transformer`생성시 다음 설정을 지정해야 합니다. 

* __instance count__ - 추론 작업을 실행할 머신의 개수
* __instance type__ - 추론 작업을 실행할 머신의 인스턴스 타입
* __output path__ - 추론 결과가 저장될 위치 

In [81]:
transform_output_folder

'batch-transform-output'

In [95]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = tree.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path,
                               assemble_with='Line',
                               accept='text/csv')

생성한 `transformer`로부터 배치 추론 씰행을 위해 `tranform()`을 호출합니다. 작업 실행시 다음 옵션들이 설정 가능합니다.

* __data_location__ - 입력 데이터의 위치
* __content_type__ - HTTP 요청을 생성할 때 사용할 콘텐츠 타입 
* __split_type__ - 입력 데이터에 대한 구분자 
* __input_filter__ - HTTP 요청을 생성할 때 제외되어야 할 컬럼(ID)에 대한 구분 (지정하지 않으면 전체 데이터를 사용함)

In [96]:
transformer.transform(data_location, content_type='text/csv', split_type='Line', input_filter='$[1:]') #? inputfilter
transformer.wait()

....................

KeyboardInterrupt: 

설정 옵션에 대한 보다 자세한 내용은 다음 [CreateTransformJob API](https://docs.aws.amazon.com/sagemaker/latest/dg/API_CreateTransformJob.html) 링크를 참조하세요.

### 결과 확인

배치 추론 작업의 결과를 S3로부터 가져온 후 결과를 확인합니다.

In [17]:
s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/iris.csv.out".format(transform_output_folder), '/tmp/iris.csv.out')
with open('/tmp/iris.csv.out') as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))

Transform results: 
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica


In [22]:
! docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [41]:
! docker images

REPOSITORY                                                                   TAG                 IMAGE ID            CREATED             SIZE
533821149268.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-decision-trees   latest              26bfbfd83383        18 hours ago        378MB
sagemaker-decision-trees                                                     latest              26bfbfd83383        18 hours ago        378MB
ubuntu                                                                       18.04               3339fde08fc3        6 days ago          63.3MB


In [ ]:
! docker images 

In [42]:
! docker run -it --name tmp1 sagemaker-decision-trees:latest

]0;root@21b9fb169e55: /opt/programroot@21b9fb169e55:/opt/program# ^C

]0;root@21b9fb169e55: /opt/programroot@21b9fb169e55:/opt/program# 

In [ ]:
! docker start tmp1
! docker attach tmp1

In [43]:
! docker run -it --name using_container 533821149268.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-decision-trees

]0;root@4f4fb61fdc80: /opt/programroot@4f4fb61fdc80:/opt/program# ^C

]0;root@4f4fb61fdc80: /opt/programroot@4f4fb61fdc80:/opt/program# 

In [45]:
! docker ps -a

CONTAINER ID        IMAGE                                                                        COMMAND             CREATED             STATUS                       PORTS               NAMES
4f4fb61fdc80        533821149268.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-decision-trees   "/bin/bash"         2 minutes ago       Exited (130) 8 seconds ago                       using_container
21b9fb169e55        sagemaker-decision-trees:latest                                              "/bin/bash"         16 minutes ago      Exited (130) 2 minutes ago                       tmp1


In [47]:
! docker start 21b9fb169e55

21b9fb169e55


In [49]:
! docker attach 21b9fb169e55

]0;root@21b9fb169e55: /opt/programroot@21b9fb169e55:/opt/program# ^C

]0;root@21b9fb169e55: /opt/programroot@21b9fb169e55:/opt/program# amazon	aws  containerd  dlami	ml


In [55]:
! docker run -it --name tmp4 sagemaker-decision-trees:latest /opt/ml && pwd

docker: Error response from daemon: OCI runtime create failed: container_linux.go:370: starting container process caused: exec: "/opt/ml": stat /opt/ml: no such file or directory: unknown.
ERRO[0000] error waiting for container: context canceled 


In [ ]:
! docker run -it --name tmp1 sagemaker-decision-trees:latest

In [136]:
!aws s3 cp {s3://sagemaker-ap-northeast-2-533821149268/to_exp_models/random_forest/} ~/SageMaker/sagemaker-byos-byocBYOCscikit_bring_your_own_krcontainer


usage: aws s3 cp <LocalPath> <S3Uri> or <S3Uri> <LocalPath> or <S3Uri> <S3Uri>
Error: Invalid argument type


In [134]:
! pwd

/home/ec2-user/SageMaker/sagemaker-byos-byoc/BYOC/scikit_bring_your_own_kr


In [142]:
!aws s3 cp {'s3://sagemaker-ap-northeast-2-533821149268/to_exp_models/random_forest'} /home/ec2-user/SageMaker/sagemaker-byos-byocBYOCscikit_bring_your_own_kr/container


Unknown options: -R


In [144]:
! ls -l ./container

total 24
-rwxrwxr-x 1 ec2-user ec2-user 1475 Mar 30 08:21 build_and_push.sh
drwxrwxr-x 2 ec2-user ec2-user 4096 Apr  2 08:08 decision_trees
-rw-rw-r-- 1 ec2-user ec2-user 1524 Mar 30 08:21 Dockerfile
drwxrwxr-x 3 ec2-user ec2-user 4096 Mar 30 08:21 local_test
-rw-rw-r-- 1 ec2-user ec2-user 5707 Mar 30 08:21 ReadMe.md


In [163]:
! mkdir ./container/random_forest

mkdir: cannot create directory ‘./container/random_forest’: File exists


In [178]:
! ls -l ./container/random_forest

total 0


In [175]:
! ls -l ./container

total 28
-rwxrwxr-x 1 ec2-user ec2-user 1475 Mar 30 08:21 build_and_push.sh
drwxrwxr-x 2 ec2-user ec2-user 4096 Apr  2 08:08 decision_trees
-rw-rw-r-- 1 ec2-user ec2-user 1524 Mar 30 08:21 Dockerfile
drwxrwxr-x 3 ec2-user ec2-user 4096 Mar 30 08:21 local_test
drwxrwxr-x 2 ec2-user ec2-user 4096 Apr  2 08:56 random_forest
-rw-rw-r-- 1 ec2-user ec2-user 5707 Mar 30 08:21 ReadMe.md


In [152]:
#! rm -r ./container/random_forest

In [170]:
#!aws s3 sync {'s3://sagemaker-ap-northeast-2-533821149268/to_exp_models/random_forest'} /home/ec2-user/SageMaker/sagemaker-byos-byocBYOCscikit_bring_your_own_kr/container/random_forest

In [ ]:
nginx.conf

predictor.py

serve

train

wsgi.py

In [180]:
!aws s3 cp --recursive {'s3://sagemaker-ap-northeast-2-533821149268/to_exp_models/random_forest/nginx.conf'} ./

In [182]:
!aws s3 sync {'s3://sagemaker-ap-northeast-2-533821149268/to_exp_models/random_forest'} /home/ec2-user/SageMaker/sagemaker-byos-byoc/BYOC/scikit_bring_your_own_kr/container/random_forest 

download: s3://sagemaker-ap-northeast-2-533821149268/to_exp_models/random_forest/nginx.conf to container/random_forest/nginx.conf
download: s3://sagemaker-ap-northeast-2-533821149268/to_exp_models/random_forest/predictor.py to container/random_forest/predictor.py
download: s3://sagemaker-ap-northeast-2-533821149268/to_exp_models/random_forest/serve to container/random_forest/serve
download: s3://sagemaker-ap-northeast-2-533821149268/to_exp_models/random_forest/train to container/random_forest/train
download: s3://sagemaker-ap-northeast-2-533821149268/to_exp_models/random_forest/wsgi.py to container/random_forest/wsgi.py


In [187]:
! pwd

/home/ec2-user/SageMaker/sagemaker-byos-byoc/BYOC/scikit_bring_your_own_kr
